### Problem statement, scope
#### 1. Objective
<!-- trying to frame problem statement, define scope -->
Design and demonstrate an **agentic workflow** capable of answering user queries related to Singapore public transport. Primary focus will be on **real-time bus arrival information**.
Agent will dynamically decide when, how to fetch data from APIs and generate responses via LLM.

#### 2. Agent responsibilities
- interpret user queries (in natural language)
- identify user intent, relevant entities
- dynamically invoke LTA APIs
- ?? incorporate constraints depending on context ?? (eg - time of the day, weather conditions?, holidays?)
- generate llm response
- use langgraph to showcase clear, modular workflow


### Tools, API Selection

#### 1. External data source - LTA DataMall APIs
agent will rely on LTA DataMall as the main source for real time Singapore public transport data

#### 2. Selected APIs
- **Bus arrival API*** : provides real time estimated arrival time for buses at given stop
- Most common user intent : 'when is the next bus arriving?'

- **Bus stops API** : provides metadata about bus stops

#### 3. Tools to support contextual reasoning
- **Time context tool** : extract current time, categorize it
- **Weather context tool** : simulate (randomize) / externally fetch weather conditions
- **Holiday/event rules**

### Agentic Workflow - LangGraph
- Decision driven workflow, stateful instead of a single LLM call.

- **Agent state components**:
  - **user_query** : user input in natural language
  - **intent** : intent inferred from user query (eg - bus arrival, service delayed etc.)
  - **entities** : bus stop (name or code?), time references
  - **context** : time of day, weather, holidays or special events
  - **planned_tools** : tools, APIs agent decided to invoke
  - **tool_results** : output returned by tools
  - **final_response** : final output generated for user

  - **Workflow Nodes** - agent workflow nodes orchestrated by LangGraph
    - **interpret input node** - analyze raw query, infer intent, extract relevant entities
    - **context enrichment node** - augment state with context not stated explicitly by user
    - **planning node** - decide which tools/api to invoke, order of tool execution
    - **tool execution node** - invoke selected tools/apis
    - **response creation node** - combine tool results+context, generate response via LLM 

### LangGraph Skeleton Implementation

In [ ]:
from typing import TypedDict, List, Dict, Any
from langgraph.graph import StateGraph, END

# agent state
# using typed dict to make value types consistent
class AgentState(TypedDict):
    user_query: str
    intent: str
    entities: Dict[str, Any]
    context: Dict[str, Any]
    planned_tools: List[str]
    tool_results: Dict[str, Any]
    final_response: str

# node to interpret user input
def interpret_input(state: AgentState) -> AgentState:
    # interpret raw user query to extract intent, entities
    # use placeholder for now
    return {
        **state,
        "intent": "bus_arrival_query",
        "entities": {}
    }